# Dataset.
Сайт: https://www.allrecipes.com, раздел everyday-cooking

Парсинг. Main. Задание начальных параметров и запуск

Выделены следующие признаки для объекта, проходящего парсинг:
- URL - ссылка - (str) 
- Title - название - (str) 
- Caterory - категория (str) (В последствии этот признак выбран целевым)
- Rating - рейтинг - (float)
- Preparation time - время подготовки - (int)
- Cook time - время приготовления - (int)
- Total time - общее время - (int)
- Servings - число порций - (int)
- Ingredientrs - ингредиенты - (str)
- Image - изображения - (str) (в таблице сохраняются пути для скачанных изображений) 

In [ ]:
import os
import re
import arff
import requests
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from urllib.parse import urljoin
from parse_site import start_parse
from difflib import SequenceMatcher
from binarize_data import binarize_data
from arff_helper import write_arff, read_tsv
from sklearn.preprocessing import MinMaxScaler
from unificate_funcs import time_to_minutes, unificate_ingredients, process_servings

tsv_filename = 'allrecipes_everyday_cooking.tsv'
arff_filename = 'allrecipes_everyday_cooking.arff'
output_csv = 'allrecipes_binarized_everyday_cooking.csv'
filter_keyword = 'everyday-cooking'

start_parse(tsv_filename, arff_filename, output_csv, filter_keyword)

Основная функция управляющая всем парсингом и последующими действиями.

In [ ]:
def start_parse(tsv_filename, arff_filename, output_csv, filter_keyword=None):
    category_links = get_category_links(filter_keyword)
    all_recipes = []
    for category_url in category_links:
        print(f"Парсинг категории: {category_url}")
        category_recipes = get_recipes_from_category_or_list(category_url)
        all_recipes.extend(category_recipes)
        print(f"Собрано {len(category_recipes)} рецептов из категории {category_url}")

    print(f"Всего рецептов собрано: {len(all_recipes)}")

    df = pd.DataFrame(all_recipes,
                      columns=['URL', 'Title', 'Category', 'Rating', 'Prep_Time_(min)',
                               'Cook_Time_(min)', 'Total_Time_(min)', 'Servings',
                               'Ingredients', 'Image_Paths'])
    df.to_csv(tsv_filename, sep='\t', index=False)

    df = read_tsv(tsv_filename)
    write_arff(df, arff_filename)

    binarize_data(arff_filename, output_csv)

Изначальной задачей было найти необходимый раздел. Ищем header на главной странице сайта, далее ищем все возможные ссылки содержащие в себе 'everyday-cooking'

In [2]:
def get_category_links(filter_keyword=None):
    url = "https://www.allrecipes.com/"
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(response.text, 'lxml')
    nav = soup.find('nav', id='mntl-header-nav_1-0')
    categories = nav.find_all('a', href=True)
    category_links = [category['href'] for category in categories if 'recipes' in category['href']]
    if filter_keyword:
        category_links = [link for link in category_links if filter_keyword in link]
    return category_links

Далее, так как структура рецептов может быть вложенной - необходимо описать взаимодействие с парсингом страниц, имеющих сами рецепты, а так же вложенные ссылки на рецепты. 

In [ ]:
def get_recipes_from_nested_list(soup):
    nested_recipes = []
    recipe_list = soup.find('div', class_='comp list-sc mntl-block article-content list')
    if recipe_list:
        content_list = recipe_list.find('div', class_='comp list-sc__content mntl-sc-page mntl-block')
        if content_list:
            list_items = content_list.find_all('div', class_='comp list-sc-item mntl-block mntl-sc-list-item')
            for item in list_items:
                title_block = item.find('span', class_='mntl-sc-block-heading__text')
                link_block = item.find('div',
                                       class_='comp mntl-sc-block allrecipes-sc-block-featuredlink mntl-sc-block-universal-featured-link mntl-sc-block-universal-featured-link--button')

                if title_block and link_block and link_block.find('a', href=True):
                    recipe_title = title_block.text.strip()
                    recipe_link = link_block.find('a', href=True)['href']

                    if 'undefinedundefined' in recipe_link:
                        print(f"Пропуск рецепта с недействительной ссылкой: {recipe_title}")
                        continue

                    print(f"Парсинг вложенного рецепта: {recipe_title}, ссылка: {recipe_link}")
                    recipe_details = get_recipe_details(recipe_link, recipe_title)
                    nested_recipes.extend(recipe_details)
                else:
                    print(f"Пропуск элемента без заголовка или ссылки.")

        else:
            print("Список контента не найден в рецепте.")
    else:
        print("Рецепт не содержит вложенных элементов.")

    return nested_recipes

In [ ]:
def get_recipes_from_category_or_list(category_url):
    recipes_data = []
    response = requests.get(category_url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(response.text, 'lxml')

    list_items = soup.find_all('div', class_='mntl-sc-block allrecipes-sc-block-heading mntl-sc-block-heading')

    if list_items:
        for list_item in list_items:
            recipe_title = list_item.find('span', class_='mntl-sc-block-heading__text').text.strip()
            recipe_link = list_item.find_next('a', href=True)['href']

            if 'undefinedundefined' in recipe_link:
                print(f"Пропуск рецепта с недействительной ссылкой: {recipe_title}")
                continue

            print(f"Список рецептов: Переход к рецепту {recipe_title}")
            recipe_details = get_recipe_details(recipe_link, recipe_title)
            recipes_data.extend(recipe_details)
    else:
        recipe_cards = soup.find_all('a', class_='mntl-card-list-items', href=True)
        for card in recipe_cards:
            recipe_url = card['href']

            if 'undefinedundefined' in recipe_url:
                print(f"Пропуск рецепта с недействительной ссылкой: {recipe_url}")
                continue

            print(f"Парсинг рецепта: {recipe_url}")
            recipe_details = get_recipe_details(recipe_url)
            recipes_data.extend(recipe_details)

    return recipes_data

Приведенные выше функции используют get_recipe_detales для парсинга с уже найденной страницы рецепта необходимых признаков и их значений

In [ ]:
def get_recipe_details(recipe_url, recipe_title="No title"):
    if 'undefinedundefined' in recipe_url:
        print(f"Пропуск недействительной ссылки: {recipe_url}")
        return []

    response = requests.get(recipe_url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(response.text, 'lxml')
    title = recipe_title if recipe_title != "No title" else (
        soup.find('h1', class_='article-heading').text.strip() if soup.find('h1',
                                                                            class_='article-heading') else 'No title')

    if "gallery" in recipe_url or soup.find('div', class_='comp list-sc'):
        print(f"Найдено вложенное меню рецептов. Парсинг вложенных рецептов.")
        return get_recipes_from_nested_list(soup)

    category = get_category_from_path(soup)

    rating = np.nan
    rating_block = soup.find('div', id='mm-recipes-review-bar_1-0')
    if rating_block:
        rating_value_block = rating_block.find('div', class_='mm-recipes-review-bar__rating')
        if rating_value_block:
            rating = rating_value_block.text.strip()

    prep_time = cook_time = total_time = servings = np.nan
    details_section = soup.find('div', class_='mm-recipes-details__content')
    if details_section:
        details_items = details_section.find_all('div', class_='mm-recipes-details__item')
        for item in details_items:
            label = item.find('div', class_='mm-recipes-details__label').text.strip()
            value = item.find('div', class_='mm-recipes-details__value').text.strip()
            if label == 'Prep Time:':
                prep_time = time_to_minutes(value)
            elif label == 'Cook Time:':
                cook_time = time_to_minutes(value)
            elif label == 'Total Time:':
                total_time = time_to_minutes(value)
            elif label == 'Servings:':
                servings = process_servings(value)

    ingredients = unificate_ingredients(parse_ingredients(soup))
    image_paths = save_recipe_images(soup, title)
    return [[recipe_url, title, category, rating, prep_time, cook_time, total_time,servings, ingredients, ', '.join(image_paths)]]

Так же функция для правильного парсинга ингредиентов:

In [ ]:
def parse_ingredients(soup):
    ingredients = []
    ingredients_div = soup.find('div', id='mm-recipes-structured-ingredients_1-0')

    if ingredients_div:
        ingredients_list = ingredients_div.find_all('li', class_='mm-recipes-structured-ingredients__list-item')
        for item in ingredients_list:
            quantity = item.find('span', attrs={'data-ingredient-quantity': 'true'})
            unit = item.find('span', attrs={'data-ingredient-unit': 'true'})
            name = item.find('span', attrs={'data-ingredient-name': 'true'})

            ingredient_text = ""
            if quantity:
                ingredient_text += quantity.text.strip() + " "
            if unit:
                ingredient_text += unit.text.strip() + " "
            if name:
                ingredient_text += name.text.strip()

            if ingredient_text.strip():
                ingredients.append(ingredient_text.strip())

    return ', '.join(ingredients) if ingredients else np.nan

Функция сохранения изображений (изображения сохраняются в папке pics/"name recipe"/img_"img num".jpg)

In [ ]:
def save_recipe_images(soup, recipe_name):
    image_folder = os.path.join('pics', recipe_name)
    os.makedirs(image_folder, exist_ok=True)
    article_content = soup.find('div', class_='loc article-content')

    if not article_content:
        print("Блок с контентом рецепта не найден.")
        return []

    image_elements = article_content.find_all('img')
    image_paths = []
    image_count = 0

    for img in image_elements:
        img_url = img.get('src')

        if not img_url:
            continue

        if 'srcset' in img.attrs:
            img_url = img['srcset'].split(',')[-1].split()[0]

        img_url = urljoin('https://www.allrecipes.com', img_url)
        img_data = requests.get(img_url).content
        img_name = f"image_{image_count + 1}.jpg"
        img_path = os.path.join(image_folder, img_name)

        with open(img_path, 'wb') as img_file:
            img_file.write(img_data)

        image_paths.append(img_path)
        image_count += 1

    if image_count == 0:
        print("Изображения не найдены для рецепта.")
    else:
        print(f"Всего сохранено изображений: {image_count} для рецепта {recipe_name}")

    return image_paths

Функция, используемая для парсинга категории из "пути" рецепта

In [ ]:
def get_category_from_path(soup):
    category = "No info"
    breadcrumbs = soup.find('ul', class_='comp mntl-universal-breadcrumbs mntl-block type--squirrel breadcrumbs')
    if breadcrumbs:
        breadcrumb_items = breadcrumbs.find_all('li', class_='comp mntl-breadcrumbs__item mntl-block')

        if len(breadcrumb_items) > 1:
            category_span = breadcrumb_items[1].find('span', class_='link__wrapper')
            if category_span:
                category = category_span.text.strip()

    return category

Однако полученные при парсинге данные требуют некоторой унификации, поэтому используются:

- Унификация времени (перевод едениц измерения в минуты)

In [ ]:
def time_to_minutes(time):
    hours = 0
    minutes = 0

    hours_match = re.search(r'(\d+)\s*hr', time)
    if hours_match:
        hours = int(hours_match.group(1))

    minutes_match = re.search(r'(\d+)\s*min', time)
    if minutes_match:
        minutes = int(minutes_match.group(1))

    total_minutes = hours * 60 + minutes
    return total_minutes

- Унификация количества числа порций сервировки

In [ ]:
def process_servings(servings_value):
    if isinstance(servings_value, str) and 'to' in servings_value:
        parts = servings_value.split('to')
        try:
            return str((float(parts[0].strip()) + float(parts[1].strip())) / 2)
        except ValueError:
            return np.nan

    if isinstance(servings_value, str):
        try:
            return str(float(servings_value.split()[0]))
        except (ValueError, IndexError):
            return np.nan

    return servings_value

- Унификация объемов и масс ингредиентов

In [ ]:
def unificate_ingredients(ingredients_string):
    if not isinstance(ingredients_string, str):
        return ""
    ingredients = ingredients_string.split(', ')
    ingredients_unificated = [unificate_ingr(ingr) for ingr in ingredients]

    return ', '.join(ingredients_unificated)

In [ ]:
def unificate_ingr(ingredient):
    ingredient = handle_fractional_numbers(ingredient)
    match = re.search(r'(\d+(\.\d+)?)\s*([a-zA-Z]+)', ingredient)

    if match:
        quantity = float(match.group(1))
        unit = match.group(3)
        converted_quantity, conversion_factor = convert_to_base_unit(quantity, unit)
        if conversion_factor:
            normalized_str = re.sub(r'(\d+(\.\d+)?)\s*([a-zA-Z]+)', f'{converted_quantity:.2f} grams', ingredient)
            return normalized_str

    return ingredient

In [ ]:
def handle_fractional_numbers(ingredient):
    for frac, dec in fraction_conversion.items():
        ingredient = ingredient.replace(frac, str(dec))

    ingredient = re.sub(r'(\d+)\s+(\d+\.\d+)', lambda match: str(float(match.group(1)) + float(match.group(2))), ingredient)

    return ingredient

In [ ]:
def convert_to_base_unit(quantity, unit):
    unit = unit.lower()
    if unit in unit_conversion:
        return quantity * unit_conversion[unit], unit_conversion[unit]
    return quantity, None

### Единицы измерения и их эквиваленты в граммах и миллилитрах

Для удобства парсинга рецептов был использован следующий словарь для преобразования единиц измерения в граммы и миллилитры:

- ounce = 28.35 грамм
- pound = 453.6 грамм
- cup = 240 мл
- tablespoon = 15 мл
- teaspoon = 5 мл
- quart = 946 мл
- liter = 1000 мл
- clove = 5 грамм


Этот словарь позволяет автоматически унифицировать различные единицы измерения в рецептах, такие как вес или объем ингредиентов. Полный список можно видеть ниже

In [10]:
from conversation_dict import unit_conversion
unit_conversion

{'ounce': 28.35,
 'ounces': 28.35,
 'pound': 453.6,
 'pounds': 453.6,
 'cup': 240,
 'cups': 240,
 'tablespoon': 15,
 'tablespoons': 15,
 'teaspoon': 5,
 'teaspoons': 5,
 'quart': 946,
 'quarts': 946,
 'liter': 1000,
 'liters': 1000,
 'milliliter': 1,
 'milliliters': 1,
 'gram': 1,
 'grams': 1,
 'pinch': 0.36,
 'pinches': 0.36,
 'clove': 5,
 'cloves': 5,
 'slice': 15,
 'slices': 15,
 'piece': 20,
 'pieces': 20,
 'can': 400,
 'cans': 400,
 'dash': 0.6,
 'dashes': 0.6,
 'fluid ounce': 29.57,
 'fluid ounces': 29.57,
 'head': 400,
 'heads': 400,
 'sheet': 5,
 'sheets': 5,
 'wrapper': 10,
 'wrappers': 10,
 'egg': 50,
 'eggs': 50,
 'stick': 113,
 'sticks': 113,
 'package': 200,
 'packages': 200,
 'bunch': 100,
 'bunches': 100,
 'leaf': 5,
 'leaves': 5,
 'sprig': 2,
 'sprigs': 2}

In [11]:
from conversation_dict import fraction_conversion
fraction_conversion

{'½': 0.5,
 '⅓': 0.3333333333333333,
 '¼': 0.25,
 '¾': 0.75,
 '⅔': 0.6666666666666666,
 '⅛': 0.125}

Дальнейшей задачей было преобразование tsv в arff файл, чтение tsv происходит при помощи функции read_tsv

In [ ]:
def read_tsv(tsv_filename):
    return pd.read_csv(tsv_filename, sep='\t')

Далее при помощи write_arff из открытого файла происходит запись в arff файл: 

In [ ]:
def write_arff(df, arff_filename):
    if os.path.exists(arff_filename):
        print(f"ARFF файл '{arff_filename}' уже существует.")
        return

    with open(arff_filename, 'w') as f:
        f.write('@relation allrecipes_recipes\n\n')

        f.write('@attribute URL string\n')
        f.write('@attribute Title string\n')
        f.write('@attribute Category string\n')
        f.write('@attribute Rating numeric\n')
        f.write('@attribute "Prep_Time_(min)" numeric\n')
        f.write('@attribute "Cook_Time_(min)" numeric\n')
        f.write('@attribute "Total_Time_(min)" numeric\n')
        f.write('@attribute Servings numeric\n')
        f.write('@attribute Ingredients string\n')
        f.write('@attribute "Image_Paths" string\n\n')

        f.write('@data\n')
        for index, row in df.iterrows():
            f.write(f'"{escape_quotes(row["URL"])}", "{escape_quotes(row["Title"])}", '
                    f'"{escape_quotes(row["Category"])}", '
                    f'{row["Rating"] if pd.notnull(row["Rating"]) else "?"}, '
                    f'{row["Prep_Time_(min)"] if pd.notnull(row["Prep_Time_(min)"]) else "?"}, '
                    f'{row["Cook_Time_(min)"] if pd.notnull(row["Cook_Time_(min)"]) else "?"}, '
                    f'{row["Total_Time_(min)"] if pd.notnull(row["Total_Time_(min)"]) else "?"}, '
                    f'{row["Servings"] if pd.notnull(row["Servings"]) else "?"}, '
                    f'"{escape_quotes(row["Ingredients"])}", "{escape_quotes(row["Image_Paths"])}"\n')

    print(f"ARFF файл '{arff_filename}' успешно создан.")

В функции write_arff используется escape_quotes для избегания ошибок связанных с тем, что в названиях могут быть кавычки, что приведет к неправильной интерпретации 

In [ ]:
def escape_quotes(s):
    if isinstance(s, str):
        return s.replace('"', '\\"')
    return s

# Предобработка
Целевым признаком выбран Category. Признаки URL, Title и Image_Paths исключены из итогового файла. Принято решение преобразовать признак Ingredients посредством one hot encoding. 

In [ ]:
def binarize_data(arff_filename, output_csv):
    with open(arff_filename, 'r') as f:
        dataset = arff.load(f)

    df = pd.DataFrame(dataset['data'], columns=[attr[0] for attr in dataset['attributes']])

    for col in df.select_dtypes(include=[object]).columns:
        if isinstance(df[col].iloc[0], bytes):
            df[col] = df[col].astype(str)
    df.fillna(df.mean(numeric_only=True), inplace=True)

    numeric_columns = df.select_dtypes(include=[float, int]).columns
    df[numeric_columns] = df[numeric_columns].round(1)

    if 'Ingredients' in df.columns:
        df['Ingredients'] = df['Ingredients'].apply(lambda x: simplify_ingredients(x.split(',')) if pd.notna(x) else [])

        unique_ingredients = set([ingredient for sublist in df['Ingredients'] for ingredient in sublist])
        similar_ingredients_map = find_similar_ingredients(list(unique_ingredients))

        df['Ingredients'] = df['Ingredients'].apply(lambda x: merge_similar_ingredients(x, similar_ingredients_map))

        ingredients_df = df['Ingredients'].str.join('|').str.get_dummies()
        ingredients_df = group_rare_ingredients(ingredients_df)

        df = pd.concat([df, ingredients_df], axis=1)
        df = df.drop('Ingredients', axis=1)

    df = df.drop(['URL', 'Title', 'Image_Paths'], axis=1)

    columns_to_normalize = ['Rating', 'Prep_Time_(min)', 'Cook_Time_(min)', 'Total_Time_(min)', 'Servings']
    df = normalize_columns(df, columns_to_normalize)

    if 'Category' in df.columns:
        df['Category'] = df['Category']

    if 'Title' in df.columns:
        df['Title'] = df['Title']

    df.to_csv(output_csv, index=False)

    print(f"Бинаризация завершена. Данные сохранены в {output_csv}. Количество признаков: {len(df.columns)}")

В процессе преобразования использовались:

In [ ]:
def normalize_columns(df, columns):
    scaler = MinMaxScaler()
    df[columns] = scaler.fit_transform(df[columns])
    return df

In [ ]:
def merge_similar_ingredients(ingredient_list, similar_map):
    return [similar_map.get(ingredient, ingredient) for ingredient in ingredient_list]

In [ ]:
def group_rare_ingredients(ingredients_df, min_frequency=5):
    ingredient_counts = ingredients_df.sum(axis=0)
    rare_ingredients = ingredient_counts[ingredient_counts < min_frequency].index

    ingredients_df['other_ingredients'] = ingredients_df[rare_ingredients].sum(axis=1)
    ingredients_df = ingredients_df.drop(columns=rare_ingredients)

    return ingredients_df

А так же для уменьшения числа признаков было принято решение объединить похожие посредством метрики Левенштейна:

In [ ]:
def simplify_ingredients(ingredients):
    simplified_ingredients = []

    for ingredient in ingredients:
        simplified = re.sub(r'\d+\s*\w*', '', ingredient).strip()
        simplified = re.sub(r'[().\/®%-]', '', simplified).strip()
        simplified_ingredients.append(simplified)
    return simplified_ingredients

In [ ]:
def find_similar_ingredients(ingredients, threshold=0.75):
    similar_map = {}
    for i in range(len(ingredients)):
        for j in range(i + 1, len(ingredients)):
            if SequenceMatcher(None, ingredients[i], ingredients[j]).ratio() > threshold:
                if len(ingredients[i]) < len(ingredients[j]):
                    similar_map[ingredients[j]] = ingredients[i]
                else:
                    similar_map[ingredients[i]] = ingredients[j]
    return similar_map

In [ ]:
def merge_similar_ingredients(ingredient_list, similar_map):
    merged_ingredients = []
    for ingredient in ingredient_list:
        merged_ingredients.append(similar_map.get(ingredient, ingredient))
    return merged_ingredients